In [1]:
'''
File name: COVIDISTRESS_cleaner.ipynb
Groupe: ADAMsLEG
Date created: 9/11/2022
Date last modified: 12/11/2022
Python Version: 3.9
'''

'\nFile name: COVIDISTRESS_cleaner.ipynb\nGroupe: ADAMsLEG\nDate created: 9/11/2022\nDate last modified: 12/11/2022\nPython Version: 3.9\n'

# COVIDiSTRESS data cleaning
In this notebook, we take the already cleaned data from the survey [COVIDiSTRESS Global Survey dataset on psychological and behavioural consequences of the COVID-19 outbreak](https://www.nature.com/articles/s41597-020-00784-9#Sec7). We then process it for our needs and output an adapted version.

In [2]:
import pandas as pd

### Set-up

In [3]:
PATH = 'data/COVIDISTRESS_unprocess_data/'

In [4]:
#getting the csv from COVIDistress 
df_april = pd.read_csv(PATH + 'COVIDiSTRESS_April_27_clean.csv', encoding= 'unicode_escape')
df_mai = pd.read_csv(PATH + 'COVIDiSTRESS_May_30_cleaned_final.csv', encoding= 'unicode_escape')
df_june = pd.read_csv(PATH + 'COVIDiSTRESS June 17.csv', encoding= 'unicode_escape')

C:\Users\josan\AppData\Local\Temp\ipykernel_11476\1405483630.py:3: DtypeWarning: Columns (55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mai = pd.read_csv(PATH + 'COVIDiSTRESS_May_30_cleaned_final.csv', encoding= 'unicode_escape')
C:\Users\josan\AppData\Local\Temp\ipykernel_11476\1405483630.py:4: DtypeWarning: Columns (54,55,56,57,58,59,60,65,66,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  df_june = pd.read_csv(PATH + 'COVIDiSTRESS June 17.csv', encoding= 'unicode_escape')


##### Constant lists

In [5]:
EUROPE_COUNTRY = ['Austria',
                  'Belgium',
                  'Bosnia and Herzegovina',
                  'Bulgaria',
                  'Croatia',
                  'Czech Republic',
                  'Denmark',
                  'Finland',
                  'France',
                  'Germany',
                  'Greece',
                  'Hungary',
                  'Ireland',
                  'Italy',
                  'Kosovo',
                  'Lithuania',
                  'Netherlands',
                  'Poland',
                  'Portugal',
                  'Serbia',
                  'Slovakia',
                  'Spain',
                  'Sweden',
                  'Switzerland',
                  'United Kingdom']

#columns that interest us for the analysis
COLUMNS_AVG = ['PSS10_avg', 'SLON3_avg']

### 1st file: data from April 27
##### Overview
We start with a slight overview of the data, in order to understand what we are going to work with.

In [6]:
df_april.head(3)

,Unnamed: 0,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,Country,...,Expl_media_6,Final_open,PSS10_avg,Lon_avg,neu,ext,ope,agr,con,SPS_avg
0,1,1058,2020-04-27 04:20:30,DA,33,Female,"College degree, bachelor, master",Some College or equivalent,Full time employed,Denmark,...,3.0,NaN,1.6,2.333333,2.000000,5.000000,4.666667,4.000000,3.666667,6.000000
1,2,1323,2020-04-27 04:17:16,NL,52,Male,"College degree, bachelor, master",Up to 12 years of school,Self-employed,Netherlands,...,4.0,Ik ben blij met de 'intelligente lockdown' zoa...,2.5,3.000000,4.666667,3.666667,5.000000,4.333333,4.666667,4.400000
2,3,801,2020-04-27 04:05:14,SK,52,Female,"College degree, bachelor, master",College degree,Full time employed,Slovakia,...,4.0,NaN,2.4,3.666667,3.000000,4.333333,4.333333,5.000000,4.666667,5.222222


In [7]:
print('This dataset contains ' + str(len(df_april)) + ' data points')

This dataset contains 115213 data points


##### Processing
We started by looking at the countries that interest us.

In [8]:
#groupe the df by country
country_groupe_april = df_april.groupby(['Country']).size()
country_groupe_april

Country
Afghanistan    110
Albania         79
Algeria         33
Andorra         14
Angola           9
              ... 
Venezuela        8
Vietnam        171
Zambia           4
Zimbabwe         4
other          218
Length: 173, dtype: int64

We can see here that we have >170 countries and that some of them only have a few data points. Hence we decided to keep only those that have >200 data points in order to have enough data per country.

In [9]:
#groupe countries that have >200 data point
country_groupe_april_filtered = country_groupe_april[country_groupe_april >= 200]

#Filters out the countries that doesn't have >200 data points
df_april_filtered = df_april[df_april['Country'].isin(country_groupe_april_filtered.index)]
country_april_over200 = df_april_filtered.groupby(['Country']).size()
print('There is ' + str(len(country_april_over200)) + ' countries that have more than 200 data points')

There is 41 countries that have more than 200 data points


We decided to orientate our evaluation to Europe countries, therefore we need to get ride of the other data from other countries

In [10]:
#change name of a columns for to coincide with the other datasets
df_april_filtered = df_april_filtered.rename(columns= {'Lon_avg' : 'SLON3_avg'})

#filters out the countries which aren't in the European List Country
df_april_filtered_EU = df_april_filtered[df_april_filtered['Country'].isin(EUROPE_COUNTRY)]
country_april_EU = df_april_filtered_EU.groupby(['Country']).size()
print('There is now only ' + str(len(country_april_EU)) + ' EU countries left')
print('And ' + str(len(df_april_filtered_EU)) + ' is the new size of the df')

There is now only 25 EU countries left
And 83525 is the new size of the df


##### Filtering NAs
Now that we have our countries of interest, we need to clean the data from any NAs in the columns of interest.

In [11]:
df_april_final = df_april_filtered_EU.dropna(subset = COLUMNS_AVG)
print('The final dataset for the month of April is of size : ' + str(len(df_april_final)))
print('There were ' + str(len(df_april_filtered_EU) - len(df_april_final)) + ' data point which were discarded')

The final dataset for the month of April is of size : 77464
There were 6061 data point which were discarded


In [12]:
df_april_final.head(3)

,Unnamed: 0,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,Country,...,Expl_media_6,Final_open,PSS10_avg,SLON3_avg,neu,ext,ope,agr,con,SPS_avg
0,1,1058,2020-04-27 04:20:30,DA,33,Female,"College degree, bachelor, master",Some College or equivalent,Full time employed,Denmark,...,3.0,NaN,1.6,2.333333,2.000000,5.000000,4.666667,4.000000,3.666667,6.000000
1,2,1323,2020-04-27 04:17:16,NL,52,Male,"College degree, bachelor, master",Up to 12 years of school,Self-employed,Netherlands,...,4.0,Ik ben blij met de 'intelligente lockdown' zoa...,2.5,3.000000,4.666667,3.666667,5.000000,4.333333,4.666667,4.400000
2,3,801,2020-04-27 04:05:14,SK,52,Female,"College degree, bachelor, master",College degree,Full time employed,Slovakia,...,4.0,NaN,2.4,3.666667,3.000000,4.333333,4.333333,5.000000,4.666667,5.222222


We now have our cleaned dataset for the data from April 27 2020

### 2nd file: data from Mai 30 2020
##### Overview
We start with a slight overview of the data, in order to understand what we are going to work with.

In [13]:
df_mai.head(3)

,ID,answered_all,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,...,Final_open,PSS10_avg,SLON3_avg,neu,ext,ope,agr,con,SPS_avg,Scale_UCLA_TRI_avg
0,1,No,180,2020-05-30 23:47:17,SAR,29,Female,"College degree, bachelor, master",Some College or equivalent,Not employed,...,NaN,2.9,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,No,3100,2020-05-29 23:30:15,UR,20,Male,"College degree, bachelor, master",None,Student,...,NaN,2.2,2.333333,2.0,5.0,5.333333,5.0,5.0,5.0,NaN
2,3,No,127,2020-05-30 22:40:15,SAR,47,Female,"Some College, short continuing education or eq...",Some College or equivalent,Self-employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
print('This dataset contains ' + str(len(df_mai)) + ' data points')

This dataset contains 125306 data points


##### Processing
We started by looking at the countries that interest us.

In [15]:
#group the df by country
country_groupe_mai = df_mai.groupby(['Country']).size()
country_groupe_mai

Country
Afghanistan    114
Albania         79
Algeria         38
Andorra         15
Angola          11
              ... 
Venezuela       11
Vietnam        173
Zambia           4
Zimbabwe         4
other          223
Length: 177, dtype: int64

We can see here that we also have >170 countries and that some of them also have only a few data points. Hence we decided to keep only those that have >200 data points in order to have enough data per country.

In [16]:
#groupe countries that have >200 data point
country_groupe_mai_filtered = country_groupe_mai[country_groupe_mai >= 200]

#Filters out the countries that doesn't have >200 data points
df_mai_filtered = df_mai[df_mai['Country'].isin(country_groupe_mai_filtered.index)]
country_mai_over200 = df_mai_filtered.groupby(['Country']).size()
print('There is ' + str(len(country_mai_over200)) + ' countries that have more than 200 data points')

There is 43 countries that have more than 200 data points


We then get ride of the other data from other countries.

In [17]:
#filters out the countries which aren't in the European List Country
df_mai_filtered_EU = df_mai_filtered[df_mai_filtered['Country'].isin(EUROPE_COUNTRY)]
country_mai_EU = df_mai_filtered_EU.groupby(['Country']).size()
print('There is now only ' + str(len(country_mai_EU)) + ' EU countries left')
print('And ' + str(len(df_mai_filtered_EU)) + ' is the new size of the df')

There is now only 25 EU countries left
And 88877 is the new size of the df


##### Filtering NAs
Now that we have our countries of interest, we need to clean the data from any NAs in the columns of interest.

In [18]:
df_mai_final = df_mai_filtered_EU.dropna(subset = COLUMNS_AVG)
print('The final dataset for the month of Mai is of size : ' + str(len(df_mai_final)))
print('There were ' + str(len(df_mai_filtered_EU) - len(df_mai_final)) + ' data point which were discarded')

The final dataset for the month of Mai is of size : 82380
There were 6497 data point which were discarded


df_mai_final.head(3)

We now have our cleaned dataset for the data from Mai 30 2020

### 2nd file: data from June 17 2020
##### Overview
We start with a slight overview of the data, in order to understand what we are going to work with.

In [19]:
df_june.head(3)

,Unnamed: 0,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,Country,...,Final_open,PSS10_avg,SLON3_avg,neu,ext,ope,agr,con,SPS_avg,Scale_UCLA_TRI_avg
0,1,777,2020-06-17 01:01:39,NL,48,Female,"College degree, bachelor, master",NaN,Full time employed,Netherlands,...,Reguliere ziekenhuiszorg is verminderd: mijn c...,2.4,2.666667,4.000000,3.666667,4.666667,4.333333,5.000000,5.1,NaN
1,2,952,2020-06-16 23:19:18,PL,36,Female,"College degree, bachelor, master",Up to 12 years of school,Full time employed,Poland,...,NaN,1.7,1.000000,2.333333,3.666667,3.000000,3.666667,4.666667,5.3,NaN
2,3,2396,2020-06-16 22:47:36,EN,45,Female,"College degree, bachelor, master",Up to 12 years of school,Full time employed,Finland,...,NaN,1.7,2.000000,2.666667,4.333333,5.000000,5.000000,5.000000,5.2,NaN


In [20]:
print('This dataset contains ' + str(len(df_june)) + ' data points')

This dataset contains 126405 data points


##### Processing
We started by looking at the countries that interest us.

In [21]:
#group the df by country
country_groupe_june = df_june.groupby(['Country']).size()
country_groupe_june

Country
Afghanistan    115
Albania         80
Algeria         38
Andorra         15
Angola          11
              ... 
Venezuela       11
Vietnam        173
Zambia           4
Zimbabwe         4
other          224
Length: 177, dtype: int64

We can see here that we also have >170 countries and that some of them also have only a few data points. Hence we decided to keep only those that have >200 data points in order to have enough data per country.

In [22]:
#groupe countries that have >200 data point
country_groupe_june_filtered = country_groupe_june[country_groupe_mai >= 200]

#Filters out the countries that doesn't have >200 data points
df_june_filtered = df_june[df_june['Country'].isin(country_groupe_june_filtered.index)]
country_june_over200 = df_june_filtered.groupby(['Country']).size()
print('There is ' + str(len(country_june_over200)) + ' countries that have more than 200 data points')

There is 43 countries that have more than 200 data points


We then get ride of the other data from other countries.

In [23]:
#filters out the countries which aren't in the European List Country
df_june_filtered_EU = df_june_filtered[df_june_filtered['Country'].isin(EUROPE_COUNTRY)]
country_june_EU = df_june_filtered_EU.groupby(['Country']).size()
print('There is now only ' + str(len(country_june_EU)) + ' EU countries left')
print('And ' + str(len(df_june_filtered_EU)) + ' is the new size of the df')

There is now only 25 EU countries left
And 89179 is the new size of the df


##### Filtering NAs
Now that we have our countries of interest, we need to clean the data from any NAs in the columns of interest.

In [24]:
df_june_final = df_june_filtered_EU.dropna(subset = COLUMNS_AVG)
print('The final dataset for the month of June is of size : ' + str(len(df_june_final)))
print('There were ' + str(len(df_june_filtered_EU) - len(df_june_final)) + ' data point which were discarded')

The final dataset for the month of June is of size : 82646
There were 6533 data point which were discarded


In [25]:
df_june_final.head(3)

,Unnamed: 0,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,Country,...,Final_open,PSS10_avg,SLON3_avg,neu,ext,ope,agr,con,SPS_avg,Scale_UCLA_TRI_avg
0,1,777,2020-06-17 01:01:39,NL,48,Female,"College degree, bachelor, master",NaN,Full time employed,Netherlands,...,Reguliere ziekenhuiszorg is verminderd: mijn c...,2.4,2.666667,4.000000,3.666667,4.666667,4.333333,5.000000,5.1,NaN
1,2,952,2020-06-16 23:19:18,PL,36,Female,"College degree, bachelor, master",Up to 12 years of school,Full time employed,Poland,...,NaN,1.7,1.000000,2.333333,3.666667,3.000000,3.666667,4.666667,5.3,NaN
2,3,2396,2020-06-16 22:47:36,EN,45,Female,"College degree, bachelor, master",Up to 12 years of school,Full time employed,Finland,...,NaN,1.7,2.000000,2.666667,4.333333,5.000000,5.000000,5.000000,5.2,NaN


We now have our cleaned dataset for the data from Mai 30 2020

### Output
We can now output our processed files

In [26]:
#Uncomment to output cleaned file
""" 
MONTHS = ['april', 'mai', 'june']
PATH_OUTPUT = 'data/'
csv_names = []
for month in MONTHS:
    csv_names += ['COVIDISTRESS_' + month + '_cleaned']
df_april_final.to_csv(PATH_OUTPUT + csv_names[0])
df_april_final.to_csv(PATH_OUTPUT + csv_names[1])
df_april_final.to_csv(PATH_OUTPUT +csv_names[2])
"""

" \nMONTHS = ['april', 'mai', 'june']\nPATH_OUTPUT = 'data/'\ncsv_names = []\nfor month in MONTHS:\n    csv_names += ['COVIDISTRESS_' + month + '_cleaned']\ndf_april_final.to_csv(PATH_OUTPUT + csv_names[0])\ndf_april_final.to_csv(PATH_OUTPUT + csv_names[1])\ndf_april_final.to_csv(PATH_OUTPUT +csv_names[2])\n"